# Webviewer

This notebook exemplifies the visualization with the webviewer tool. It can also be served through port forwarding so that the data can be visualized remotely. For more information on how to dop this please check the Readme.

> **important**: Please run the Example datasets first before running this notebook, as it expects the tree and mesh outputs to exist! 


In [ ]:
import panel as pn
pn.extension('vtk')
pn.extension()
import holoviews as hv
hv.extension('bokeh', width=100)

import os
import pandas as pd
import numpy as np
import param
import matplotlib.pyplot as plt
from holoviews import opts
from holoviews.streams import Selection1D
from inter_view.color import clipped_plasma_r, plot_colorbar
from improc.io import parse_collection, DCAccessor
from lstree.lineage.plot import plot_tree
import time
from io import BytesIO
import argparse

DCAccessor.register()


opts.defaults(
    opts.Layout(sizing_mode='stretch_both'),
    opts.Overlay('tree', yaxis=None, show_title=False, framewise=True, responsive=True, min_width=1000, min_height=1000),
    opts.Points('tree', size=3, cmap=clipped_plasma_r, framewise=True, colorbar=True, colorbar_position='right', responsive=True, colorbar_opts={'bar_line_width':0, 'width':15}),
    opts.Segments(line_width=1, color='black', framewise=True, responsive=True),
    opts.VLine(line_dash='dashed', color='grey', line_width=1, responsive=True, framewise=True))

parser = argparse.ArgumentParser()
parser.add_argument("--basedir", help="data directory", nargs=1)
args, unknown = parser.parse_known_args()

## parse meshes and trees

First we parse all of the mesh and tree outputs we got from the two example datasets

In [ ]:
# when using this viewer as a notebook, specify base data directory here. Currently the folder with the Example datasets is set. When using remotely, the basedir argument must be provided.
if args.basedir is None:
    basedir = '../example/data'
else:
    basedir = args.basedir[0]


# create/update symlink to access mesh data from javascript
static_basedir = 'static_data'
if os.path.exists(static_basedir):
    os.unlink(static_basedir)
    
os.symlink(basedir, static_basedir, target_is_directory=True)

df_grid = parse_collection(os.path.join(static_basedir, '{dataset}/{subdir}/{f2}T{time:04d}.vti'), ['dataset', 'subdir', 'time'])
df_mesh = parse_collection(os.path.join(static_basedir, '{dataset}/{subdir}/{f2}T{time:04d}.vtp'), ['dataset', 'subdir', 'time'])
df_vtk = pd.concat([df_grid, df_mesh])
df_h5 = parse_collection(static_basedir + '/{dataset}/agg_features.{ext}', ['dataset'])

# Load first sample and get list of available node/edge attributes
Important: here we assume the same list for the entire experiment

In [ ]:
dfn = pd.read_hdf(df_h5.iloc[0:1].dc.path[0], 'nodes')
dfe = pd.read_hdf(df_h5.iloc[0:1].dc.path[0], 'edges')

NODE_ATTRIBUTES = sorted(dfn.select_dtypes(['int', 'float']).columns.tolist() + ['selection', 'neighbor_trace'])
EDGE_ATTRIBUTES = sorted(list( set(dfe.select_dtypes(['int', 'float']).columns) - {'node_start', 'node_stop'}))

# tree viewer

In [ ]:
class TreeViewer(param.Parameterized):
    '''Tree viewer for interactive visualization with Holoviews'''
    time_interval = param.Number(10/60)
    h5_path = param.Parameter()
    dfe = param.DataFrame()
    dfn = param.DataFrame()
    
    node_color = param.ObjectSelector(default='dodgerblue', objects=['dodgerblue'] + NODE_ATTRIBUTES)
    node_size = param.ObjectSelector(default='fixed', objects=['fixed', None] + NODE_ATTRIBUTES)
    
    edge_color = param.ObjectSelector(default='black', objects=['black'] + EDGE_ATTRIBUTES)
    edge_size = param.ObjectSelector(default='fixed', objects=['fixed'] + EDGE_ATTRIBUTES)
    
    tooltips = param.List(['mamut_id', 'mamut_t'])
    
    hv_tree = param.Parameter(hv.Overlay([hv.Scatter([])]))
    
    last_tap = param.Parameter(hv.streams.SingleTap(transient=True), instantiate=True)
    last_clicked_mamut_t = param.Number(1)
    tree_update_counter = param.Integer(0)
    
    selection = param.Parameter(Selection1D(), instantiate=True)
    selection_update_counter = param.Integer(0)
    
    def __init__(self, *args, **kargs):
        super().__init__(*args, **kargs)
        
        self._load_tree()
    
    def _set_time_interval(self):
        '''estimate time interval upon loading new data'''
        self.time_interval = np.median(np.diff(np.sort(self.dfn.time.unique())))
    
    @param.depends('h5_path', watch=True)
    def _load_tree(self):
        if self.h5_path:
            
            try:
                self.dfn = pd.read_hdf(self.h5_path, 'nodes')#.iloc[:100]
                self.dfn['selection']  = 0
                self.dfn['neighbor_trace']  = 0
                self.dfe = pd.read_hdf(self.h5_path, 'edges')#.iloc[:0]
                self._set_time_interval()
            except KeyError as e:
                self.dfn = None
                
            self.tree_update_counter += 1
    
    @param.depends('selection.index', watch=True)
    def update_selection_column(self):
        if len(self.selection.index) > 0:
            self.dfn['selection']  = 0
            self.dfn['selection'].iloc[self.selection.index] = 1
            self.selection_update_counter += 1
            
            if 'selection' in [self.node_color, self.node_size, self.edge_color, self.edge_size]:
                self.tree_update_counter += 1
    
    @param.depends('tree_update_counter', 'node_color', 'node_size', 'edge_color', 'edge_size')
    def _plot_tree(self):
        
        try:
            # dynamic update of color,size,etc. does not work when it depends on vdims --> we are forced to rebuild 
            # the plot everytime...
            if self.node_size == 'fixed':
                node_size = 5
            else:
                node_size = self.node_size

            if self.edge_size == 'fixed':
                edge_size = 2
            else:
                edge_size = self.edge_size

            self.hv_tree = plot_tree(self.dfn,
                                     self.dfe,
                                     tooltips=self.tooltips,
                                     node_size=node_size,
                                     node_color=self.node_color,
                                     edge_width=edge_size,
                                     edge_color=self.edge_color,
                                     min_node_size=2,
                                     max_node_size=30,
                                     min_edge_width=1,
                                     max_edge_width=9,)

            # link the last clicked position
            self.last_tap.source = self.hv_tree.Tree.II

            # add selection stream and attach callback to update sample/image selection
            self.selection.source=self.hv_tree.Tree.II

            return self.hv_tree
        
        except Exception as e:
            # tree does not exist or does not contain the requried features
            self.hv_tree = hv.Overlay([hv.Scatter([])])
            self.dfn = None
            self.dfe = None
            return self.hv_tree
    
    # somehow does not work when combined with vtk pane
    @param.depends('last_tap.x', watch=True)
    def _update_last_clicked_timepoint(self):
        if self.last_tap.x is not None:
            self.last_clicked_mamut_t = int(round(self.last_tap.x / self.time_interval))
    
    def widgets(self):
        return pn.WidgetBox(self.param.node_color,
                            self.param.node_size,
                            self.param.edge_color,
                            self.param.edge_size)
    
    def plot(self):    
        tree_dmap = hv.DynamicMap(self._plot_tree)#.opts(responsive=True)
        return tree_dmap.opts(xlabel='time').opts(opts.Points(tools=['hover', 'box_select', 'lasso_select', 'tap']))
    
    def panel(self):
        # dummy plot somehow needed forthe selection to continue updating after dynamic map is refreshed
        self.dummy = hv.Points([], group='tree')
        return pn.Row(self.widgets(), self.dummy * self.plot())


# ds_id = 6
# tv = TreeViewer(h5_path=df_h5.dc[ds_id].dc.path[0],
#                 node_color='nuclei_volume')

# tv.panel().servable()

# HTML Panel
Needed for sending arbitrary javascript code.

In [ ]:
class JSCode():
    '''Creates a (hidden) dummy HTML panel that allows 
    sending arbitrary javascript code'''
    
    def __init__(self):
        self.dummy = pn.pane.HTML("", style={'padding':'0px', 'visibility':'hidden'}, width=0, height=0, margin=0)
    
    def run(self, code):
        self.dummy.object = ''
        self.dummy.object = """<script type="text/javascript">{}</script>""".format(code)

def make_feature_lut(cmap, ids, values, bounds):
    ids = np.array(ids)[:,None]
    values = (values-bounds[0]) / (bounds[1]-bounds[0])
    colors = cmap(values)[:,:3]
    return np.concatenate([ids, colors], axis=1).tolist()

# Organoid viewer

In [ ]:
class OrganoidViewer(param.Parameterized):
    '''Visualization of 3D segmetnation meshes.'''
    
    spacing = param.Tuple((1,1,1))
    
    nuclei_mesh = param.Parameter()
    cell_mesh = param.Parameter()
    rgb_grid = param.Parameter()
    
    features = param.DataFrame()
    features_bounds = param.Dict()

    cell_representation = param.ObjectSelector(default="Wireframe", objects=['Wireframe', 'Surface'])
    nuclei_representation = param.ObjectSelector(default="Surface", objects=['Wireframe', 'Surface'])
    
    cell_opacity = param.Number(0.1, bounds=(0,1), step=0.01)
    nuclei_opacity = param.Number(1., bounds=(0,1), step=0.01)
    
    cell_color = param.ObjectSelector(default=None, objects=[None] + NODE_ATTRIBUTES)
    nuclei_color = param.ObjectSelector(default=None, objects=[None] + NODE_ATTRIBUTES)
    cell_lut = param.List([[0, 1., 1., 1.]])
    nuclei_lut = param.List([[0, 0., 0.56470588, 1.]])
    
    cmap = param.Parameter(clipped_plasma_r)
    
    def __init__(self, *args, **kargs):
        super().__init__(*args, **kargs)
        
        self.jscode = JSCode()
        
        self.nuclei_opacity_wg = pn.panel(self.param.nuclei_opacity)
        self.cell_opacity_wg = pn.panel(self.param.cell_opacity)
        self.nuclei_representation_wg = pn.panel(self.param.nuclei_representation)
        self.cell_representation_wg = pn.panel(self.param.cell_representation)
        
        with open('vtkjs-panel.html', 'r') as f:
            html_vtkjs_panel = f.read()
            
        # set paths to load on first render
        if self.features is not None:
            self._update_nuclei_color()
            self._update_cell_color()
            code = self._generate_update_js_code()
        else:
            code = ''
        html_vtkjs_panel = html_vtkjs_panel.replace('{{init_code}}',code)

        self.html_pane = pn.pane.HTML(html_vtkjs_panel, min_height=800, min_width=800, sizing_mode='stretch_both')
        self._set_callbacks()

        
    def clear_renderer(self):
        self.jscode.run("""renderer.removeActor(imageActorK);
                           renderer.removeActor(imageActorI);
                           renderer.removeActor(imageActorJ);
                           renderer.removeActor(nucleiActor);
                           renderer.removeActor(cellActor);
                           render();""")
    
    def _generate_update_js_code(self):
        
        code = """
               nucleiPath = '{nuclei_path}';
               var colorArray = JSON.parse('{nuclei_lut}');
               updateNucleiMesh(colorArray, {nuclei_opacity}, '{nuclei_repr}');
               
               cellPath = '{cell_path}';
               var colorArray = JSON.parse('{cell_lut}');
               updateCellMesh(colorArray, {cell_opacity}, '{cell_repr}');
               
               imgPath = '{grid_path}'
               updateImage();
               """.format(
                    nuclei_path=self.nuclei_mesh,
                    nuclei_lut=self.nuclei_lut,
                    nuclei_opacity=self.nuclei_opacity_wg.value,
                    nuclei_repr=self.nuclei_representation_wg.value,
                    cell_path=self.cell_mesh,
                    cell_lut=self.cell_lut,
                    cell_opacity=self.cell_opacity_wg.value,
                    cell_repr=self.cell_representation_wg.value,
                    grid_path=self.rgb_grid)

        
        return code
        
    def load(self, features, nuclei_mesh, cell_mesh, rgb_grid=None, features_bounds=None):
        self.features = features
        self.nuclei_mesh = nuclei_mesh
        self.cell_mesh = cell_mesh
        self.rgb_grid = rgb_grid
                
        if features_bounds is not None:
            self.features_bounds = features_bounds
        
        self._update_nuclei_lut()
        self._update_cell_lut()
        
        code = self._generate_update_js_code()
        self.jscode.run(code)
        
    def update_features(self, features):
        if self.nuclei_color in self.features.columns:
            nuclei_changed = not self.features[self.nuclei_color].equals(features[self.nuclei_color])
        else:
            nuclei_changed = False
            
        if self.cell_color in self.features.columns:
            cell_changed = not self.features[self.cell_color].equals(features[self.cell_color])
        else:
            cell_changed = False
        
        self.features = features
        if nuclei_changed:
            self._update_nuclei_color()
            
        if cell_changed:
            self._update_cell_color()
        
    def _set_callbacks(self):
        
        self.nuclei_opacity_wg.jslink(self.html_pane , code={'value':"""
            nucleiActor.getProperty().setOpacity(source.value);
            render();
            """})
        
        self.cell_opacity_wg.jslink(self.html_pane , code={'value':"""
            cellActor.getProperty().setOpacity(source.value);
            render();
            """})
        
        self.nuclei_representation_wg.jslink(self.html_pane , code={'value':"""
            if(source.value=="Wireframe"){{
                nucleiActor.getProperty().setRepresentationToWireframe();
            }}else if(source.value=="Surface"){{
                nucleiActor.getProperty().setRepresentationToSurface();
            }}
            render();
            """})
        
        self.cell_representation_wg.jslink(self.html_pane , code={'value':"""
            if(source.value=="Wireframe"){{
                cellActor.getProperty().setRepresentationToWireframe();
            }}else if(source.value=="Surface"){{
                cellActor.getProperty().setRepresentationToSurface();
            }}
            render();
            """})
    
    def _update_nuclei_lut(self):
        if self.nuclei_color:
            self.nuclei_lut = make_feature_lut(cmap=self.cmap,
                                   ids=self.features['timepoint_id']+1,
                                   values=self.features[self.nuclei_color],
                                   bounds=self.features_bounds[self.nuclei_color])
        else:
            self.nuclei_lut = [[0, 0., 0.56470588, 1.]]
    
    @param.depends('nuclei_color', watch=True)
    def _update_nuclei_color(self):
        
        self._update_nuclei_lut()
        
        self.jscode.run("""
            var colorArray = JSON.parse('{}')
            updateLUT(nucleiLookupTable, colorArray, nucleiPolydata);
        """.format(str(self.nuclei_lut)))
        
        
    def _update_cell_lut(self):
        if self.cell_color:
            self.cell_lut = make_feature_lut(cmap=self.cmap,
                                   ids=self.features['timepoint_id']+1,
                                   values=self.features[self.cell_color],
                                   bounds=self.features_bounds[self.cell_color])
        else:
            self.cell_lut = [[0, 1., 1., 1.]]
    
    @param.depends('cell_color', watch=True)
    def _update_cell_color(self):
        
        self._update_cell_lut()
        
        self.jscode.run("""
            var colorArray = JSON.parse('{}')
            updateLUT(cellLookupTable, colorArray, nucleiPolydata);
        """.format(str(self.cell_lut)))
    
    def _bg_hook(self, plot, element):
        '''Directly access bokeh figure and set selection'''
        plot.state.border_fill_color = "#f5f5f5"
    
    @param.depends('nuclei_color')
    def _nuclei_colorbar(self):
        if self.nuclei_color is None:
            return None
        else:
            cbar = plot_colorbar(clipped_plasma_r, self.features_bounds[self.nuclei_color], orientation='horizontal', backend='bokeh')
            return cbar.opts(hooks=[self._bg_hook])

    @param.depends('cell_color')
    def _cell_colorbar(self):
        if self.cell_color is None:
            return None
        else:
            cbar = plot_colorbar(clipped_plasma_r, self.features_bounds[self.cell_color], orientation='horizontal', backend='bokeh')
            return cbar.opts(hooks=[self._bg_hook])

    def _nuclei_widgets(self):
        return pn.WidgetBox(self.nuclei_opacity_wg,
                            self.param.nuclei_color,
                            self._nuclei_colorbar,
                            self.nuclei_representation_wg,)
        
    def _cell_widgets(self):
        return pn.WidgetBox(self.cell_opacity_wg,
                            self.param.cell_color,
                            self._cell_colorbar,
                            self.cell_representation_wg,)
        
    def widgets(self):
        return pn.Row(self._nuclei_widgets(),
                      self._cell_widgets(),
#                       pn.Column(self._slice_widgets, self._colorbars)
                     )
    
    @property
    def vtkpan(self):
        return pn.Row(self.html_pane, self.jscode.dummy, sizing_mode='stretch_both')
    
    def panel(self):
        return pn.Column(self.vtkpan,
                         self.widgets())


# mamut_t = 323

# ds_id = 3
# h5_path = df_h5.dc[ds_id].dc.path[0]
# dfn = pd.read_hdf(h5_path, 'nodes')
# dfe = pd.read_hdf(h5_path, 'edges')

# features_bounds = dfn.agg(['min', 'max']).to_dict('list')

# db = OrganoidViewer(features=dfn.set_index('mamut_t').loc[mamut_t:mamut_t],
#                     features_bounds=features_bounds,
#                     nuclei_mesh=df_vtk.dc[ds_id, 'nuclei_mesh', mamut_t].dc.path[0],
#                     cell_mesh=df_vtk.dc[ds_id, 'cell_mesh', mamut_t].dc.path[0],
#                     rgb_grid=df_vtk.dc[ds_id, 'rgb_grid', mamut_t].dc.path[0])

# # db = OrganoidViewer()

# panel = db.panel().servable()
# panel



In [ ]:
# mamut_t = 421
# ds_id = 2

# db.load(features=dfn.set_index('mamut_t').loc[mamut_t:mamut_t],
#         nuclei_mesh=df_vtk.dc[ds_id, 'nuclei_mesh', mamut_t].dc.path[0],
#         cell_mesh=df_vtk.dc[ds_id, 'cell_mesh', mamut_t].dc.path[0],
#         rgb_grid=df_vtk.dc[ds_id, 'rgb_grid', mamut_t].dc.path[0],
#         features_bounds=features_bounds)

# Dashboard
Putting it all together. At the end it will start the interactive panel on the notebook.

In [ ]:
import time
from inter_view.io import CollectionHandler

class OrganoidDashboard(param.Parameterized):
    
    df_vtk = param.DataFrame(pd.DataFrame())
    df_h5 = param.DataFrame(pd.DataFrame())
    
    dataset = param.ObjectSelector(default=0, objects=[0])
    timepoint = param.Integer(100, bounds=(1, 100))
    
    tree_viewer = param.Parameter(TreeViewer(node_color='nuclei_volume'), instantiate=False)
    organoid_viewer = param.Parameter(OrganoidViewer(nuclei_color='nuclei_volume'))
    
    def __init__(self, *args, **kargs):
        super().__init__(*args, **kargs)
        
        self.param.dataset.objects = self.df_h5.index.tolist()
        if self.dataset not in self.param.dataset.objects:
            self.dataset = self.param.dataset.objects[0]
            
        self.export_button = pn.widgets.FileDownload(label='export', callback=self._export_callback, filename='tmp.svg')
            
        self._update_tree_viewer()
        self._update_sample()
            
    @param.depends('timepoint', watch=True)
    def _update_sample(self):
        dfn = self.tree_viewer.dfn
        self.organoid_viewer.load(features=dfn.set_index('mamut_t').loc[self.timepoint:self.timepoint],
                                  nuclei_mesh=self.df_vtk.dc[self.dataset, 'nuclei_mesh', self.timepoint].dc.path[0],
                                  cell_mesh=self.df_vtk.dc[self.dataset, 'cell_mesh', self.timepoint].dc.path[0],
                                  rgb_grid=self.df_vtk.dc[self.dataset, 'rgb_grid', self.timepoint].dc.path[0])
        
    @param.depends('dataset', watch=True)
    def _update_tree_viewer(self):
        self.tree_viewer.h5_path = self.df_h5.dc[self.dataset].dc.path[0]
        
        if self.tree_viewer.dfn is not None:
            self._compute_features_bounds()
            
            t_bounds = tuple(self.tree_viewer.dfn.mamut_t.agg(['min', 'max']))
            self.param.timepoint.bounds = t_bounds
            self.timepoint = (t_bounds[0] + t_bounds[1])//2
            self._update_sample()
        else:
            self.organoid_viewer.clear_renderer()
        
    def _compute_features_bounds(self):
        self.organoid_viewer.features_bounds = self.tree_viewer.dfn.agg(['min', 'max']).to_dict('list')
        self.organoid_viewer.features_bounds['selection'] = [0,1]
        self.organoid_viewer.features_bounds['neighbor_trace'] = [0,2]
    
    @param.depends('tree_viewer.selection_update_counter', watch=True)
    def _update_feature_cmap(self):
        dfn = self.tree_viewer.dfn
        features = dfn.set_index('mamut_t').loc[self.timepoint:self.timepoint]
        self.organoid_viewer.update_features(features)
    
    @param.depends('tree_viewer.last_clicked_mamut_t', watch=True)
    def _select_timepoint(self):
        self.timepoint = self.tree_viewer.last_clicked_mamut_t
    
    def export_mesh(self):
        code = '''
                renderWindow.captureImages()[0].then(
                  (image) => {{
                    var download = document.createElement('a');
                    download.href = image;
                    download.download = 'DS{}_T{}_N-{}_C-{}_mesh.png';
                    download.click();
                  }}
                );
            '''
        
        code = code.format(self.dataset, self.timepoint, self.organoid_viewer.nuclei_color, self.organoid_viewer.cell_color)
        
        self.organoid_viewer.jscode.run(code)
    
    def export_plot(self):
        try:
            hv.extension('matplotlib', logo=False)

            if self.tree_viewer.node_size == 'fixed':
                node_size = 5
            else:
                node_size = self.tree_viewer.node_size

            if self.tree_viewer.edge_size == 'fixed':
                edge_size = 2
            else:
                edge_size = self.tree_viewer.edge_size

            hv_tree = plot_tree(self.tree_viewer.dfn,
                                 self.tree_viewer.dfe,
                                 tooltips=[],
                                 node_size=node_size * 10,
                                 node_color=self.tree_viewer.node_color,
                                 edge_width=edge_size,
                                 edge_color=self.tree_viewer.edge_color,
                                 min_node_size=2,
                                 max_node_size=30,
                                 min_edge_width=1,
                                 max_edge_width=9,
                                 backend='matplotlib')

            hv_tree.opts(opts.Overlay('tree', show_frame=False, yaxis=None, show_title=False, aspect=0.7, framewise=True),
                         opts.Points('tree', cmap=clipped_plasma_r, framewise=True),
                         opts.Segments('tree', color='black', cmap=clipped_plasma_r, framewise=True))

            layout = [hv_tree.opts(sublabel_size=0)]

            cbar_img = np.linspace(0, 1, 64)[:,None]
            if self.tree_viewer.node_color is not None:
                cbar = plot_colorbar('clipped_plasma_r', self.organoid_viewer.features_bounds[self.tree_viewer.node_color])
                layout.append(cbar)

            if self.organoid_viewer.nuclei_color is not None:
                cbar = plot_colorbar('clipped_plasma_r', self.organoid_viewer.features_bounds[self.organoid_viewer.nuclei_color])
                layout.append(cbar)

            if self.organoid_viewer.cell_color is not None:
                cbar = plot_colorbar('clipped_plasma_r', self.organoid_viewer.features_bounds[self.organoid_viewer.cell_color])
                layout.append(cbar)

            buff = BytesIO()

            hv.save(hv.Layout(layout).opts(framewise=True, fig_size=800, tight=True, aspect_weight=True, axiswise=True), buff, fmt='pdf')
            buff.seek(0)
            hv.extension('bokeh', logo=False)
            
        except Exception as e:
            print(e)
            hv.extension('bokeh', logo=False)
        
        return buff
    
    def _export_callback(self):
        self.export_mesh()
        buff = self.export_plot()
        self.export_button.filename = 'DS{}_color-{}_tree.pdf'.format(self.dataset,
                                                                      self.tree_viewer.node_color)
        return buff
        
    def panel(self):
        timepoint_vline = hv.VLine(x=self.tree_viewer.time_interval * self.timepoint)
        
        plots = pn.Row(timepoint_vline * self.tree_viewer.plot(),
                       self.organoid_viewer.vtkpan,
                       sizing_mode='stretch_both')
        
        widgets = pn.Row(pn.Column(pn.WidgetBox(self.param.dataset,
                                                self.param.timepoint),
                                   self.export_button,
                                   # TODO remove dummy button when panel bug fixed https://github.com/holoviz/panel/issues/1920
                                   pn.widgets.Button(width=0, margin=0)),
                         self.tree_viewer.widgets(),
                         self.organoid_viewer.widgets())
        
        # js link timepoint slider to vertical line
        code = '''glyph.location = source.value * {}'''.format(self.tree_viewer.time_interval)
        widgets[0][0][1].jslink(timepoint_vline, code={'value': code})

        return pn.Column(plots, widgets, sizing_mode='stretch_both').servable()

db = OrganoidDashboard(df_h5=df_h5,
                       df_vtk=df_vtk)

panel = db.panel()
panel

# First neighbors
Initial idea of how to calculate first neighbors. Keeping in mind that currently the dashboard does not allow the back-and-forth transfer of information in an easy way, such that selecting a node for further calculation of not yet existing feature is currently not possible in an interactive way. For the time being the way around this is to run the cells below first and then rerun the dashboard.

In [ ]:
# def trace_back_neighbors(tree, node_id):
#     # TODO check that delaunay_neighbors exist
    
#     tree.set_all_nodes_attribute('neighbor_trace', 0)
    
#     def label_ancestors(node_id):
#         current_node = tree.nodes[node_id]
#         current_node['neighbor_trace'] = 2
        
#         neighbors_ids = current_node['cell_neighbors']
#         if isinstance(neighbors_ids, list):
#             for nb_id in neighbors_ids:
#                 tree.nodes[nb_id]['neighbor_trace'] = 1

#         for p_id in tree.predecessors(node_id):
#             label_ancestors(p_id)

#     label_ancestors(node_id)
    
    
# from lstree.lineage.plot import tree_to_dataframe, plot_tree
# from lstree.lineage.ltree import LineageTree

# # needed once per dataset
# dfn = db.tree_viewer.dfn
# dfe = db.tree_viewer.dfe
# tree = LineageTree.from_dataframe(dfn, dfe, 'mamut_t')

In [ ]:
# trace_back_neighbors(tree, 1782)

# dfnb, dfeb = tree_to_dataframe(tree)
# dfn['neighbor_trace'] = dfnb['neighbor_trace']

# db.tree_viewer.dfn = dfn
# db.tree_viewer.tree_update_counter += 1
# db._update_feature_cmap()